In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import datetime

In [2]:
customers = pd.read_csv('olist_customers_dataset.csv')

In [3]:
orders = pd.read_csv('olist_orders_dataset.csv')

In [4]:
order_items = pd.read_csv('olist_order_items_dataset.csv')

In [233]:
customers.shape

(99441, 5)

In [234]:
df_customers.customer_unique_id.nunique()
#уникальных пользователей

96096

In [235]:
customers.customer_id.nunique()
#уникльных заказов

99441

In [236]:
orders.isna().sum()
#Не везде есть даты, что логично: не все заказы успешно завершены

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [237]:
orders.groupby("order_status", as_index = False).agg({"customer_id": "count"})
#взглянем на распределение заказов по статусам

,order_status,customer_id
0,approved,2
1,canceled,625
2,created,5
3,delivered,96478
4,invoiced,314
5,processing,301
6,shipped,1107
7,unavailable,609


In [238]:
order_items.order_id.nunique()
#где ещё 775 чеков?

98666

In [239]:
order_items.order_item_id.nunique()
#максимальное колво позиций в одном чеке

21

In [240]:
order_items.groupby("order_id", as_index = False).agg({"order_item_id": "count"}).sort_values("order_item_id", ascending=False).head()
#смотрим, сколько позиций товара в каждом чеке: от 1 до 21

,order_id,order_item_id
50137,8272b63d03f5f79c56e9e4120aec44ef,21
10459,1b15974a0141d54e36626dca3fdc731a,20
65715,ab14fdcfbe524636d65ee38360e22ce8,20
60941,9ef13efd6949e4573a18964dd1bbe7f5,15
25583,428a2f660dc84138d969ccd69a0ab6d5,15


# Задание 1. Сколько у нас пользователей, которые совершили покупку только один раз?

### Как понять вообще, что пользователь совершил покупку? Давайте взглянем на статусы и подумаем, как их соотнести с фактом покупки. created — создан - очевидно, человек просто набрал карзину товаров и нажал "заказать", купил ли он это? Если посмотреть на все статусы, то мы видем целый жизненный цикл заказа и делаем вывод, что нажав заказат далеко не уедешь. approved — подтверждён - значит, сайт увидел заказ, пока рано говорить о совершённой сделке invoiced — выставлен счёт - вооот,тут уже две стороны сработались и идут дальше, но если заказ не оплатить, то покупа е свершится. Идём дальше. processing — в процессе сборки заказа - вот теперь мы имеем такую картину: кто-то нажал заказать, мы подтвердили, что видим, попросили оплатить, т.к. сл. статус "shipped", предполагаем, что оплата прошла успешно и мы деньги получили, теперь можем смело говорить, что покупка совершна. Дальше отгрузка и доставка мб вообще не в нашей юрисдиции, м.б. мы отпрвляем почтой или пользуемся услугами иной логистической фирмы. shipped — отгружен со склада delivered — доставлен пользователю unavailable — недоступен canceled — отменёнИтого, будем считать, что все заказы со статусами: processing, shipped, delivered - оплачены и не отменены, соответственно, пользователь совершил покупку.

In [6]:
customers_orders = customers.merge(orders)

In [245]:
customer_one_buy = customers_orders.query("order_status == ['processing', 'shipped', 'delivered']") \
                .groupby("customer_unique_id", as_index=False) \
                .agg({"customer_zip_code_prefix": "count"}) \
                .query("customer_zip_code_prefix == 1").shape[0]

In [246]:
customer_one_buy

91814

# Задание 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?
### В первую очередь находим все отмененые, недоступные (Я посчитал что именно это является не доставкой товара) Далее приводим дату к нужным форматам Выводим график по недоставки по месяцам

In [248]:
pd.set_option('mode.chained_assignment', None) # Почему без этого не работал вызов pd.to_datetime

In [247]:
#Недоставленные согласно статусу заказа
orders.query("order_status != 'delivered'").shape[0]

2963

In [249]:
#Недоставленные, т.к. отсутствует дата доставки - выводим их
orders[orders.order_delivered_customer_date.isna()].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaN,2018-06-28 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaN,NaN,2017-10-03 00:00:00
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaN,2018-02-07 00:00:00


In [250]:
orders[orders.order_delivered_customer_date.isna()].order_status.unique()

array(['invoiced', 'shipped', 'processing', 'unavailable', 'canceled',
       'delivered', 'created', 'approved'], dtype=object)

### У нас присутсвуют все виды статусов заказа, предположим, что чсть недоставлены по логичным причинам: не подтверждён, не выствлен счёт, не собран, не отгружен со склада или банально ещё не довезли, т.е. просто ещё не завершён. Также логично, что некоторые заказы на определённом этапе были отменены или стали недоступы. Т.о. один статус заказа "delivered" - не имеющий даты доставки, мы тое исключим, возможно, какой-то сбой. Такие заказы мы не будем учитывать.

In [252]:
orders[orders.order_delivered_customer_date.isna()].query("order_status != 'delivered'").shape[0]

2957

### 2957 заказов недоставлены по разным причинам. Мы взяли все заказы без даты доставки и исключили из полученных заказы со статусом "доставлен".Но что, если мы вернёмся к первому варианту и пойдём обратным путём: сперва возьмём все заказы со статусом, отличным от "доставлен" и оттуда удалм все заказы, где оказалась дата доставки.

In [254]:
orders.query("order_status != 'delivered'")[orders.query("order_status != 'delivered'").order_delivered_customer_date.isna()].shape[0]

2957

### Всё сошлось, т.к. 6 заказов без даты доставки имеют статус "отменён"

In [260]:
#создадим новый ДФ с недоставлнными заказами
no_delivered = orders[orders.order_delivered_customer_date.isna()].query("order_status != 'delivered'")

In [263]:
no_delivered['order_purchase_timestamp'] = pd.to_datetime(no_delivered['order_purchase_timestamp'])

In [264]:
#приведём даты к месяцам, т.к. нас интересуют данным в разрезе месяца
no_delivered['order_purchase_timestamp'] = no_delivered['order_purchase_timestamp'].dt.strftime('%Y-%m')

In [266]:
#посчитаем кол-во недоставленных заказов по каждой причине за каждый месяц, далее среднее значение для каждой причины
no_delivered \
    .groupby(["order_purchase_timestamp", "order_status"], as_index=False) \
    .agg({"customer_id": "count"}) \
    .groupby("order_status") \
    .agg({"customer_id": "mean"}) \
    .round() \
    .sort_values("customer_id") \
    .reset_index()

,order_status,customer_id
0,approved,1.0
1,created,2.0
2,invoiced,15.0
3,processing,16.0
4,canceled,26.0
5,unavailable,29.0
6,shipped,48.0


# Задание 3. По каждому товару определить, в какой день недели товар чаще всего покупается. 

In [269]:
#сделаем ДФ с покупками и датами
df = orders.merge(order_items)

In [270]:
#псмотрим, что выло
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


In [271]:
#Рашее мы уже определили какие статусы мы считаем покупкой, оставим только их
df_up = df.query("order_status == ['processing', 'shipped', 'delivered']")

In [273]:
df_up['order_purchase_timestamp'] = pd.to_datetime(df_up['order_purchase_timestamp'])

In [274]:
# Единственная колонка с датами - дата создания заказа, определим по ней день недели
df_up.order_purchase_timestamp = df_up.order_purchase_timestamp.dt.day_name()

In [285]:
df_up = df_up \
    .groupby(["product_id", "order_purchase_timestamp"], as_index=False) \
    .agg({"customer_id": "count"}) \
    .sort_values(by="customer_id", ascending=False) \
    .drop_duplicates(subset='product_id') 

SpecificationError: Column(s) ['customer_id'] do not exist

In [287]:
df_up = df_up.drop('customer_id', axis=1)

In [286]:
df_up

,product_id,order_purchase_timestamp
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday
21738,ab9cf155f8280c3739b09fe341a185e8,Tuesday
21751,abafccbe06074c732dceed6675d5a7ac,Wednesday
21750,abaf51d7a67cc363c8da7ae77d15ecdb,Friday
21749,abaefce0c17047c6829ff4078f2005d0,Sunday
...,...,...
10866,5486b0f9faa264ae6cbbd2bf230bbb00,Friday
10865,54860a1533edf6c65d974441713ead8e,Tuesday
10864,5483dc3f61a1d385ccc49cec3a560617,Sunday
10863,547ef2af00ca493818dd33eb5216b22e,Friday


# Задание 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.

In [7]:
customers_orders_items = customers_orders.merge(order_items)
#обединили все три ДФ в один

In [289]:
customers_orders_items['order_purchase_timestamp'] = pd.to_datetime(customers_orders_items['order_purchase_timestamp'])

In [290]:
#приведём даты к месяцам, т.к. нас интересуют данные в разрезе месяца
customers_orders_items.order_purchase_timestamp = customers_orders_items.order_purchase_timestamp.dt.strftime('%Y-%m')

In [291]:
customers_orders_items_proba = customers_orders_items \
    .groupby(["customer_unique_id", "order_purchase_timestamp"], as_index=False) \
    .agg({"customer_state":"count"})
#выводим по каждому пользователю кол-во покупок в каждый месяц

In [292]:
customers_orders_items_proba["Mon"] = customers_orders_items_proba.order_purchase_timestamp
#делаем ещё 1 столбец с датой

In [293]:
customers_orders_items_proba["Mon"] = pd.to_datetime(customers_orders_items_proba["Mon"]).dt.month
#превращаем его в номер месяца

In [294]:
def week(mon):
        if mon == 2:
            x = 28
        elif mon == 4:
            x = 30
        elif mon == 6:
            x = 30
        elif mon == 9:
            x = 30
        elif mon == 11:
            x = 30
        else:
            x = 31
        return x 
    
#Сделали функцию, которая скажет солько дней в каждо из месяцев

In [295]:
customers_orders_items_proba.Mon = customers_orders_items_proba.Mon.apply(week)
#применили

In [303]:
customers_orders_items_proba

,customer_unique_id,order_purchase_timestamp,customer_state,Mon,peer_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,1,4.428571,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,1,4.428571,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03,1,4.428571,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,1,4.428571,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11,1,4.285714,0.233333
...,...,...,...,...,...
97314,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06,2,4.285714,0.466667
97315,fffea47cd6d3cc0a88bd621562a9d061,2017-12,1,4.428571,0.225806
97316,ffff371b4d645b6ecea244b27531430a,2017-02,1,4.000000,0.250000
97317,ffff5962728ec6157033ef9805bacc48,2018-05,1,4.428571,0.225806


In [296]:
customers_orders_items_proba.Mon = (customers_orders_items_proba.Mon / 7)
#мне кажетс или я иду по сложному пути? в datetime есть функция, котора может сразу сказать, сколько в месяце дней?
#ну, я по крайней мере умею пользоваться циклами и создавать функции. Ну, если не считать того, что or работает не так, как хотелось бы

In [298]:
customers_orders_items_proba["peer_week"] = (customers_orders_items_proba.customer_state / customers_orders_items_proba.Mon)

In [300]:
customers_orders_items_proba = customers_orders_items_proba.drop('tyt_nazvanie_metriki', axis=1)

In [301]:
customers_orders_items_proba.head()

,customer_unique_id,order_purchase_timestamp,customer_state,Mon,peer_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,1,4.428571,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,1,4.428571,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03,1,4.428571,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,1,4.428571,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11,1,4.285714,0.233333


# Задание 5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц.

### Вспомним как рассчитывается Retention: Retention дня X — это N% процентов пользователей, которые вернутся к продукту в день X. Иными словами, если в какой-то конкретный день (день 0) пришло 100 новых пользователей, а на 1-ый день вернулось 15, то Retention 1-го дня составит 15 / 100 = 15%.

In [401]:
customers_orders_5 = customers_orders
#создадим датасет под задачу

### Итого, будем считать, что все заказы со статусами: processing, shipped, delivered - оплачены и не отменены, соответственно, пользователь совершил покупку.

In [402]:
customers_orders_5 = customers_orders_5.dropna(axis='index', how='any', subset=['order_approved_at'])
#Удаляем пропуски

In [403]:
customers_orders_5["cohort"] = customers_orders_5.groupby('customer_unique_id')['order_approved_at'].transform('min')
#Создадим колонку с датой первой покупки

In [404]:
#приведём даты к месяцам, т.к. нас интересуют данные в разрезе месяца
customers_orders_5.order_purchase_timestamp = pd.to_datetime(customers_orders_5.order_purchase_timestamp).dt.strftime('%Y-%m')
customers_orders_5.order_approved_at = pd.to_datetime(customers_orders_5.order_approved_at).dt.strftime('%Y-%m')

In [405]:
customers_orders_5['first_buy'] = pd.to_datetime(customers_orders_5['cohort']).dt.strftime('%Y-%m')

In [406]:
customers_orders_5 = customers_orders_5.query("order_status == ['processing', 'shipped', 'delivered']") \
    .query("'2017-01' <= cohort <= '2017-12'") \
    .groupby(["cohort", "order_approved_at", 'first_buy'], as_index=False) \
    .agg({"customer_unique_id": "nunique"})
#фильтруем по условиям, группируем даты и считаем кол-во уникальных айди

In [409]:
customers_orders_5["first_buy"] = customers_orders_5["first_buy"].astype('datetime64[M]')
customers_orders_5["cohort"] = customers_orders_5["cohort"].astype('datetime64[M]')
customers_orders_5["order_approved_at"] = customers_orders_5["order_approved_at"].astype('datetime64[M]')

In [415]:
customers_orders_5['period'] = customers_orders_5.order_approved_at - customers_orders_5.first_buy
#считаем месяц покупки, начинаем с нулевого - месяц первой покупки

In [417]:
customers_orders_5['period'] = customers_orders_5['period'] / np.timedelta64(1,'M')
customers_orders_5['period'] = customers_orders_5['period'].round().astype('int') 

In [423]:
customers_orders_5

,cohort,order_approved_at,first_buy,customer_unique_id,period
0,2017-01-01,2017-01-01,2017-01-01,1,0
1,2017-01-01,2017-01-01,2017-01-01,1,0
2,2017-01-01,2017-01-01,2017-01-01,1,0
3,2017-01-01,2017-01-01,2017-01-01,1,0
4,2017-01-01,2017-01-01,2017-01-01,1,0
...,...,...,...,...,...
37098,2017-11-01,2018-01-01,2017-11-01,1,2
37099,2017-11-01,2017-11-01,2017-11-01,1,0
37100,2017-11-01,2017-11-01,2017-11-01,1,0
37101,2017-11-01,2017-11-01,2017-11-01,1,0


In [433]:
customers_orders_5_up = customers_orders_5.pivot_table(index='cohort', columns = 'period', values = 'customer_unique_id', aggfunc='sum')
#считаем

In [434]:
customers_orders_5_up

period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19
cohort,,,,,,,,,,,,,,,,,,,
2017-01-01,705.0,4.0,2.0,1.0,3.0,1.0,3.0,1.0,1.0,NaN,2.0,2.0,5.0,3.0,1.0,1.0,2.0,3.0,1.0
2017-02-01,1661.0,4.0,4.0,3.0,7.0,2.0,4.0,3.0,2.0,4.0,2.0,5.0,3.0,3.0,2.0,1.0,1.0,4.0,NaN
2017-03-01,2577.0,13.0,9.0,9.0,9.0,4.0,4.0,8.0,8.0,3.0,9.0,3.0,6.0,3.0,4.0,6.0,2.0,4.0,NaN
2017-04-01,2286.0,14.0,5.0,4.0,7.0,6.0,8.0,6.0,8.0,4.0,5.0,2.0,2.0,1.0,2.0,2.0,5.0,NaN,NaN
2017-05-01,3527.0,19.0,18.0,14.0,11.0,12.0,15.0,6.0,9.0,9.0,10.0,12.0,7.0,2.0,7.0,9.0,NaN,NaN,NaN
2017-06-01,3102.0,15.0,11.0,12.0,8.0,10.0,12.0,7.0,4.0,7.0,10.0,11.0,5.0,4.0,6.0,NaN,NaN,NaN,NaN
2017-07-01,3768.0,20.0,13.0,8.0,13.0,7.0,12.0,4.0,7.0,9.0,9.0,11.0,5.0,10.0,NaN,NaN,NaN,NaN,NaN
2017-08-01,4135.0,28.0,15.0,11.0,16.0,22.0,11.0,11.0,5.0,7.0,10.0,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-01,4081.0,27.0,20.0,14.0,17.0,11.0,9.0,9.0,13.0,7.0,11.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [435]:
round(customers_orders_5_up.div(customers_orders_5_up.iloc[:,0], axis = 0).mul(100), 2)[2].idxmax()
#выводим когорту с максимаьным значениеем на третий месяц (индекс 2!)

Timestamp('2017-05-01 00:00:00')

# Задание 6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю,monetary от 1780 до 3560 рублей в неделю.

In [8]:
customers_orders_6 = customers_orders_items
#создадим датасет для 6 задачи

In [9]:
customers_orders_6 = customers_orders_6.dropna()
#Удалим все пропуски

In [439]:
# order_approved_at - дата оплаты, значит продажа состоялсь, будем отталкиватся от уже имеющейся логики

In [10]:
customers_orders_6.order_approved_at = pd.to_datetime(customers_orders_6.order_approved_at)

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
last_day = customers_orders_6.order_approved_at.max()

### Допустим, мы выгрузили отчёт вчера, т.о. в строчке выше мы нашли вчерашнюю дату, добавим день, чтобы у нас получилось условное СЕГОДНЯ.

In [13]:
last_day

Timestamp('2018-08-29 15:10:26')

In [14]:
delta = datetime.timedelta(days=1)

In [15]:
today = last_day + delta

In [16]:
today

Timestamp('2018-08-30 15:10:26')

# Создаём таблицу RFM
# R - время от последней покупки пользователя до текущей даты 
# F - суммарное количество покупок у пользователя за всё время 
# M - стоимость покупок за всё время

In [18]:
RFM = customers_orders_6 \
    .groupby('customer_unique_id') \
    .agg({'order_approved_at': lambda x: (today - x.max()).days, 'product_id': lambda x: len(x), 'price': lambda x: x.sum()}) \
    .rename(columns={'order_approved_at': 'R', 'product_id': 'F', 'price': 'M'})

In [19]:
RFM

,R,F,M
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,112,1,129.90
0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,18.90
0000f46a3911fa3c0805444483337064,537,1,69.00
0000f6ccb0745a6a4b88665a16c9f078,321,1,25.99
0004aac84e0df4da2b147fca70cf8255,288,1,180.00
...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,447,2,1570.00
fffea47cd6d3cc0a88bd621562a9d061,262,1,64.89
ffff371b4d645b6ecea244b27531430a,568,1,89.90


In [20]:
# Мы будем разбивать на диапазон от 1 до 5: до 20, 20-40, 40-60, 60-80 и выше 80 %

In [21]:
RFM_seg = RFM.quantile(q=[0.20, 0.40, 0.60, 0.80]).to_dict()

In [22]:
# Напишем функцию, которая сегментирует покупателей
def to_seg (a,b,c):
    if a <= c[b][0.20]:
        return 1
    elif a <= c[b][0.40]:
        return 2
    elif a <= c[b][0.60]:
        return 3
    elif a <= c[b][0.80]:
        return 4
    else:
        return 5

In [23]:
# Применяем функцию для каждого кластера
RFM['R_seg'] = RFM['R'].apply(to_seg, args=('R', RFM_seg, ))
RFM['F_seg'] = RFM['F'].apply(to_seg, args=('F', RFM_seg, ))
RFM['M_seg'] = RFM['M'].apply(to_seg, args=('M', RFM_seg, ))

In [24]:
# Склеиваем итог в одно поле
RFM['RFMseg'] = RFM.R_seg.map(str) + RFM.F_seg.map(str) + RFM.M_seg.map(str)

In [25]:
RFM

,R,F,M,R_seg,F_seg,M_seg,RFMseg
customer_unique_id,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,112,1,129.90,2,1,4,214
0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,18.90,2,1,1,211
0000f46a3911fa3c0805444483337064,537,1,69.00,5,1,2,512
0000f6ccb0745a6a4b88665a16c9f078,321,1,25.99,4,1,1,411
0004aac84e0df4da2b147fca70cf8255,288,1,180.00,4,1,5,415
...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,447,2,1570.00,5,5,5,555
fffea47cd6d3cc0a88bd621562a9d061,262,1,64.89,3,1,2,312
ffff371b4d645b6ecea244b27531430a,568,1,89.90,5,1,3,513


# Клиенты по RFM: 555 — Лояльные и активные: покупали недавно, покупают часто и много платят; 111 — Отток: 
# покупали давно, покупают редко и мало платят; R=[1, 2], F=[4, 5], M=[4, 5] — Лояльные, потерявшие активность: 
# покупали давно, но часто и много платили